### Purpose

Decompose changes in Australian employment levels over 5 yearly intervals due to:
* australian born
* migrants that arrived in the last five years, and 
* migrants that were already in Australia 5 years ago.

The 5 year period reflects that ABS labour force data identifies overseas born employed by time since arrival in 5 year intervals.

### Data

The LM7 datacube from [ABS 6291.0.55.001 (Labour Force, Detailed)](http://www.abs.gov.au/ausstats/abs@.nsf/mf/6291.0.55.001) contains monthly data on the numbers of:
* emloyed full-time
* employed part-time
* unemployed looked for full-time work
* unemployed looked for only part-time work
* not in the labour force (nilf)

by
* the number of years since arrival for migrants (in 5 year intervals up to 20 years, and > 20 years)
* gender
* place of birth (Australia, main english speaking countries, other than main enslish speaking countries and 'Not Stated / Inadequately Described / Born at sea')
* state

This analysis uses additional derived data from this dataset. The loaded dataframe includes:
* employed_total (sum of full-, and part-time employed)
* labor_force (sum of employed_total and and unemployed)
* population (sum of labor_force and nilf)
* COB (where place of birth is mapped to 'Australia', 'overseas' and 'Unkown')


### Deriving Australian born and migrant contributions to 5 year employment changes

Identifying the contributions of australian born, recent arrivals and established migrants can be derived by:

\begin{align}
\Delta E_5 & = E_{t} - E_{t-5} \\
& = (aus\_born_{t} + migrant_{t}) - (aus\_born_{t-5} + migrant_{t-5}) \\
& = (aus\_born_{t} - aus\_born_{t-5}) + (migrant_{t} - migrant_{t-5}) \\
& = \Delta aus\_born_5 + \Delta migrant_5
\end{align}

Where, in any given month t,  $E$ is the total number employed, $aus\_born$ is the number of australian born employed, $migrant$ is the number of overseas born employed.

$t-5$ refers to the month 5 years prior to the month $t$.

The change in employment levels of migrants, $\Delta migrant_5$, can be seperated into addtions from recent arrivals and changes in employment levels for established migrants.

\begin{equation}
\Delta migrant_5 = migrant_{arrived\_in\_last\_5\_years} + \Delta migrant_{arrived\_more\_than\_5\_years\_ago}
\end{equation}

That is, the decomposition of changes in 5 yearly total employment levels is:

\begin{equation}
\Delta E_5 = \Delta aus\_born_5 + migrant_{arrived\_in\_last\_5\_years} + \Delta migrant_{arrived\_more\_than\_5\_years\_ago}
\end{equation}

As the LM7 data cube contains data on the number of people employed by whether australian born or overseas born, together with 5 yearly arrival intervals for overseas born, $\Delta migrant_{arrived\_more_than\_5\_years\_ago}$  can be derived by substitution:

\begin{equation}
\Delta migrant_{arrived\_more\_than\_5\_years\_ago} = \Delta E_5 - \Delta aus\_born_5 - migrant_{arrived\_in\_last\_5\_years}
\end{equation}

### Calculations

#### Required libraries

In [1]:
import pandas as pd

from pathlib import Path


#### Get LM7 data

In [2]:
# Set the file path to the folder containing the data

# Assume the LM7 dataset is in the same folder as this notebook
# data_folder = Path('.')

# The path for my file structure for data sets and should be ignored
data_folder = Path(f'{Path.home()}/Documents/Analysis/Australian economy/Data/ABS')


In [3]:
# The data contained in sheet 'Data 1' in the LM7 datacube has been extracted, the additional items defined above calculated (such as employment_total, etc)
# and stored as a parquet file
# This statement reads in that created datafile
# You could do this direcctly from the LM7 with pd.read_excel() statement: see last cell in this notebook for an example

df = pd.read_parquet(data_folder / 'LM7.parquet')

df.tail(3)

/Users/christopher/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/Users/christopher/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/Users/christopher/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


,sex,MESC,elapsed_years_since_arrival,state,employed_full_time,employed_part_time,unemployed_looked_full_time,unemployed_looked_part_time_only,nilf,COB,labor_force,employed_total,population
date,,,,,,,,,,,,,
2018-12-31,Females,Not Stated / Inadequately Described / Born at sea,Not stated / Inadequately described / Born at sea,Tasmania,0.00,0.00,0.00,0.00,3.81,unknown,0.00,0.00,3.81
2018-12-31,Females,Not Stated / Inadequately Described / Born at sea,Not stated / Inadequately described / Born at sea,Northern Territory,0.00,0.00,0.00,0.00,1.92,unknown,0.00,0.00,1.92
2018-12-31,Females,Not Stated / Inadequately Described / Born at sea,Not stated / Inadequately described / Born at sea,Australian Capital Territory,0.00,0.00,0.00,0.00,1.32,unknown,0.00,0.00,1.32


In [4]:
# Check the data read in
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70373 entries, 1991-01-31 to 2018-12-31
Data columns (total 13 columns):
sex                                 70373 non-null object
MESC                                70373 non-null object
elapsed_years_since_arrival         70373 non-null object
state                               70373 non-null object
employed_full_time                  70373 non-null float64
employed_part_time                  70373 non-null float64
unemployed_looked_full_time         70373 non-null float64
unemployed_looked_part_time_only    70373 non-null float64
nilf                                70373 non-null float64
COB                                 70373 non-null object
labor_force                         70373 non-null float64
employed_total                      70373 non-null float64
population                          70373 non-null float64
dtypes: float64(8), object(5)
memory usage: 7.5+ MB


In [4]:
(df
                  .groupby([df.index, 'elapsed_years_since_arrival'])['employed_total']
                  .sum()
                  .unstack('elapsed_years_since_arrival')
                  .drop(columns=['Not stated / Inadequately described / Born at sea'])
                  .sort_index(axis=1, ascending=False)
)
# #                   .reindex(labels=arrived_order, axis='columns')
#                   .assign(total = lambda x: x.sum(axis='columns'))
#                   .rename_axis(None, axis='columns')
# ).diff(60)

elapsed_years_since_arrival,Born in Australia,Arrived within last 5 years,Arrived 5-9 years ago,Arrived 20 or more years ago,Arrived 15-19 years ago,Arrived 10-14 years ago
date,,,,,,
1991-01-31,"5,710.67",240.08,205.80,"1,030.70",227.00,227.02
1991-02-28,"5,760.22",241.76,210.86,"1,038.47",233.13,224.35
1991-03-31,"5,764.85",262.07,216.76,"1,026.49",231.62,213.60
1991-04-30,"5,757.59",265.37,218.26,"1,031.50",232.52,211.97
1991-05-31,"5,742.82",271.69,213.77,"1,019.89",232.39,223.15
1991-06-30,"5,736.70",262.77,210.60,"1,020.43",228.62,218.94
1991-07-31,"5,653.47",261.13,212.85,996.39,233.28,225.87
1991-08-31,"5,692.37",275.97,207.12,996.33,222.26,226.65
1991-09-30,"5,753.18",287.98,209.83,"1,001.03",229.22,231.99


In [5]:
# check no 'countries' missed
df.COB.unique()

[overseas, Australia, unknown]
Categories (3, object): [overseas, Australia, unknown]

#### Australian born & migrant contribution to employment growth

In [8]:
def make_employed_by_duration(df, month=6):
    '''
    A function to extract employment levels for Aus. born, and OS born by time in Australia into a simple matrix from the LM7 datacube 
    
    Parameters:
    -----------
        df: the LM7 dataset (ie sheet: Data 1 from LM7 loaded in a dataframe)
        month: integer or None
            the month to use (eg 6 for financial) if doing annual calculations, if None then return all data
        
    Returns
    -------
        employed: pandas dataframe
    '''
    
    # Remove unknown COB
    idx = df.MESC != 'Not Stated / Inadequately Described / Born at sea'  # or idx = df.COB != 'unknown'


    arrived_order = ['Born in Australia',
                     'Arrived within last 5 years',
                     'Arrived 5-9 years ago',
                     'Arrived 10-14 years ago',
                     'Arrived 15-19 years ago',
                     'Arrived 20 or more years ago',
                     'total'
                    ]


    employed = (df.loc[idx]
                  .groupby(['date', 'elapsed_years_since_arrival'])['employed_total']
                  .sum()
                  .unstack('elapsed_years_since_arrival')
                  .drop(columns=['Not stated / Inadequately described / Born at sea'])
                  .sort_index(axis=1, ascending=False)
                  .reindex(labels=arrived_order, axis='columns')
                  .assign(total = lambda x: x.sum(axis='columns'))
                  .rename_axis(None, axis='columns')
        )

    if month is None:
        return employed
    else:
        idx = employed.index.month == month
        return employed[idx]



In [9]:
month = 11 #8 # use 6 for  analysis on an Australian financial year basis; use 12 for calendar, or any month as suits your analysis

employed = make_employed_by_duration(df, month=month)
employed.tail()

,Born in Australia,Arrived within last 5 years,Arrived 5-9 years ago,Arrived 10-14 years ago,Arrived 15-19 years ago,Arrived 20 or more years ago,total
date,,,,,,,
2014-11-30,"8,203.07",552.03,696.32,393.98,285.64,"1,448.12","11,579"
2015-11-30,"8,457.76",543.94,661.22,433.44,355.13,"1,479.17","11,931"
2016-11-30,"8,417.88",608.23,694.28,506.45,393.61,"1,430.19","12,051"
2017-11-30,"8,649.29",632.69,741.53,543.72,404.26,"1,476.52","12,448"
2018-11-30,"8,730.16",678.67,690.07,693.67,403.96,"1,539.80","12,736"


In [27]:
685.40 / 12836

0.053396696790277344

#### Contribution in level terms

In [10]:
# if the 'employed' dataframe data is on an annual basis, set time_delta to 5 (years).
# else if 'employed' is on a monthly basis, set the time_delta to 60 (months == 5 years)
# Comment out the unused time_delta below

if month is not None:
    # employed has annual year data
    time_delta = 5
else:
    # employed has monthly data
    time_delta = 60

idx = ['Born in Australia', 'total' ]

delta = (employed[idx]
             .diff(time_delta)
        )



delta_order = ['Born in Australia',
               'Arrived within last 5 years',
               'arrived_more_than_5_years',
               'total'
              ]


delta = (pd
             .concat([delta, employed['Arrived within last 5 years']], axis='columns')
             .assign(arrived_more_than_5_years = lambda x: x.total - x['Born in Australia'] - x['Arrived within last 5 years'])
             .reindex(labels=delta_order, axis='columns')
        )
         

delta.tail(7)

,Born in Australia,Arrived within last 5 years,arrived_more_than_5_years,total
date,,,,
2012-11-30,371.70,628.86,-157.17,843.38
2013-11-30,162.93,559.53,-33.47,688.99
2014-11-30,294.44,552.03,-94.57,751.90
2015-11-30,349.41,543.94,-104.58,788.76
2016-11-30,311.29,608.23,-105.64,813.88
2017-11-30,514.86,632.69,-75.27,"1,072.27"
2018-11-30,663.60,678.67,-51.06,"1,291.21"


#### Contribution in percentage terms

In [11]:
idx = ['Born in Australia', 'Arrived within last 5 years', 'arrived_more_than_5_years']

delta_share = (delta[idx]
                   .divide(delta.total, axis='rows') * 100
              )
       
(delta_share
     .dropna(axis='index', how='any')
     .round(0)
     .astype(int)
     .tail(21)
)

,Born in Australia,Arrived within last 5 years,arrived_more_than_5_years
date,,,
1998-11-30,78,28,-6
1999-11-30,74,31,-5
2000-11-30,71,45,-15
2001-11-30,72,36,-7
2002-11-30,74,31,-5
2003-11-30,74,32,-6
2004-11-30,72,34,-6
2005-11-30,69,31,0
2006-11-30,67,34,-1


#### How to: read in LM7 as an excel file

In [8]:
# label defintions

col_names = {'Month': 'date',
             'Sex': 'sex',
             'Main English-speaking countries': 'MESC',
             'Elapsed years since arrival': 'elapsed_years_since_arrival',
             'State and territory (STT): ASGS (2011)': 'state',
             "Employed full-time ('000)": 'employed_full_time',
             "Employed part-time ('000)": 'employed_part_time',
             "Unemployed looked for full-time work ('000)": 'unemployed_looked_full_time',
             "Unemployed looked for only part-time work ('000)": 'unemployed_looked_part_time_only',
             "Not in the labour force (NILF) ('000)": 'nilf',
             }

OSB = {'Main English-speaking countries': 'overseas',
       'Other than main English-speaking countries': 'overseas',
       'Australia (includes External Territories)': 'Australia',
       'Not Stated / Inadequately Described / Born at sea': 'unknown'
       }

idx_labor_force = ['employed_full_time', 'employed_part_time', 'unemployed_looked_full_time',
                   'unemployed_looked_part_time_only']

In [9]:
%%time
# get data
# The top 3 rows in the sheet 'Data 1' of the LM7 notebook should be unmerged, otherwise, it will take several minutes to read in the data (as opposed to ~6s on my machine machine)

df = (pd
          .read_excel(data_folder / 'LM7.xlsx',
                      usecols='A:J',
                      sheet_name='Data 1',
                      skiprows=3,
                      parse_dates=[0], infer_datetime_format=True,
                      )
          .rename(columns=col_names)
          # derive additional variables
          .assign(date=lambda x: x.date + pd.offsets.MonthEnd(0))
          .assign(COB=lambda x: x.MESC.map(OSB))
          .assign(labor_force=lambda x: x[idx_labor_force].sum(axis=1))
          .assign(employed_total=lambda x: x.employed_full_time + x.employed_part_time)
          .assign(population=lambda x: x.nilf + x.labor_force)
          .set_index('date')
          )

for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].astype('category')
        
df.tail()

CPU times: user 8.39 s, sys: 75.8 ms, total: 8.46 s
Wall time: 8.04 s


In [11]:
df.tail()

,sex,MESC,elapsed_years_since_arrival,state,employed_full_time,employed_part_time,unemployed_looked_full_time,unemployed_looked_part_time_only,nilf,COB,labor_force,employed_total,population
date,,,,,,,,,,,,,
2018-12-31,Females,Not Stated / Inadequately Described / Born at sea,Not stated / Inadequately described / Born at sea,South Australia,0.00,0.00,0.00,0.00,15.12,unknown,0.00,0.00,15.12
2018-12-31,Females,Not Stated / Inadequately Described / Born at sea,Not stated / Inadequately described / Born at sea,Western Australia,0.00,0.00,0.00,0.00,27.58,unknown,0.00,0.00,27.58
2018-12-31,Females,Not Stated / Inadequately Described / Born at sea,Not stated / Inadequately described / Born at sea,Tasmania,0.00,0.00,0.00,0.00,3.81,unknown,0.00,0.00,3.81
2018-12-31,Females,Not Stated / Inadequately Described / Born at sea,Not stated / Inadequately described / Born at sea,Northern Territory,0.00,0.00,0.00,0.00,1.92,unknown,0.00,0.00,1.92
2018-12-31,Females,Not Stated / Inadequately Described / Born at sea,Not stated / Inadequately described / Born at sea,Australian Capital Territory,0.00,0.00,0.00,0.00,1.32,unknown,0.00,0.00,1.32
